In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("br_mme_consumo_energia_eletrica_uf.csv")
df.head()

,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo
0,2004,1,TO,Total,NaN,65876
1,2004,1,BA,Total,NaN,1444451
2,2004,1,PR,Total,NaN,1596274
3,2004,1,RS,Total,NaN,1780912
4,2004,1,GO,Total,NaN,630624


In [3]:
df.describe

<bound method NDFrame.describe of         ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
0      2004    1       TO        Total                  NaN    65876
1      2004    1       BA        Total                  NaN  1444451
2      2004    1       PR        Total                  NaN  1596274
3      2004    1       RS        Total                  NaN  1780912
4      2004    1       GO        Total                  NaN   630624
...     ...  ...      ...          ...                  ...      ...
38875  2023   12       BA  Residencial            5932740.0   718461
38876  2023   12       PA  Residencial            2614106.0   478726
38877  2023   12       AC  Residencial             245699.0    58301
38878  2023   12       RN  Residencial            1390080.0   210119
38879  2023   12       SC  Residencial            2719791.0   655389

[38880 rows x 6 columns]>

In [7]:
df.isnull().sum()

ano                    0
mes                    0
sigla_uf               0
tipo_consumo           0
numero_consumidores    0
consumo                0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
def dataframe_to_sql(df, table_name, file_name):
    # Começar a criação do script SQL
    sql_script = f"CREATE TABLE {table_name} (\n"
    
    # Adicionar os tipos de dados das colunas
    for col in df.columns:
        dtype = str(df[col].dtype)
        if 'int' in dtype:
            sql_type = 'INTEGER'
        elif 'float' in dtype:
            sql_type = 'REAL'
        else:
            sql_type = 'TEXT'
        sql_script += f"    {col} {sql_type},\n"
    
    # Remover a última vírgula e adicionar o fechamento do CREATE TABLE
    sql_script = sql_script.rstrip(',\n') + "\n);\n\n"
    
    # Adicionar os INSERTs
    for _, row in df.iterrows():
        values = ', '.join([f"'{str(v)}'" if isinstance(v, str) else str(v) for v in row])
        sql_script += f"INSERT INTO {table_name} VALUES ({values});\n"
    
    # Salvar o script SQL em um arquivo
    with open(file_name, 'w') as f:
        f.write(sql_script)

    print(f"Arquivo SQL '{file_name}' criado com sucesso.")


dataframe_to_sql(df, 'consumo_energia_eletrica', 'init.sql')


Arquivo SQL 'init.sql' criado com sucesso.


O que foi feito:

Simplesmente um dropna para retirada de dados nulos, não foram muitos e não parecem impactar tanto nos resultados.
A maior parte dos dados nulos era do ano de 2004, provavelmente por falta de monitoração, ou seja, o dropna não afeta a predição de valores acima de 2004.

O database então foi convertido em um init.sql, que é adicionado à um container mysql
A tabela no container será utlizada nas predições.